In [44]:
## experiments on MobileHART model using non-PD and PD datasets

In [1]:
import pandas as pd
import numpy as np
import model
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import classification_report
from collections import defaultdict


2024-02-28 23:25:39.387249: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-28 23:25:39.387274: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-28 23:25:39.388057: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-28 23:25:39.391986: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-28 23:25:39.848992: W tensorflow/compiler/tf2

In [2]:
#read and load data
columns = ["AccelX(g)_leg","AccelY(g)_leg","AccelZ(g)_leg","GyroX(¬∞/s)_leg","GyroY(¬∞/s)_leg","GyroZ(¬∞/s)_leg","AccelX(g)_hand","AccelY(g)_hand","AccelZ(g)_hand","GyroX(¬∞/s)_hand","GyroY(¬∞/s)_hand","GyroZ(¬∞/s)_hand","AccelX(g)_chest","AccelY(g)_chest","AccelZ(g)_chest","Sample(V)","Activity_Label"]
df = pd.read_csv("/home/jupyter-saswar/HealthySubs_Modeling/healthy_smote.csv", header=0, names=columns)


In [3]:
#factorize labels
labels = pd.factorize(df['Activity_Label'])[0]

#removing target variable
feature_columns = df.columns[:-1] 
data = df[feature_columns].values

In [4]:
# define input shape and activity count as per your dataset
input_shape = (128, 16)
activityCount = 33  

#HART = model.HART(input_shape, activityCount)
MobileHART = model.mobileHART_XS(input_shape, activityCount)

2024-02-28 23:26:01.936418: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-28 23:26:01.936624: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-28 23:26:01.956561: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [5]:
def create_overlapping_windows(data, window_size, step_size, features):
    segments = []
    for start in range(0, len(data) - window_size + 1, step_size):
        segment = data[start:start + window_size, :features]
        segments.append(segment)
    return np.array(segments)

#defining parameters
window_size = 128  
step_size = 64    
num_features = 16 

reshaped_data = create_overlapping_windows(data, window_size, step_size, num_features)


def assign_labels_to_windows(labels, window_size, step_size):
    window_labels = []
    for start in range(0, len(labels) - window_size + 1, step_size):
        window_label = np.bincount(labels[start:start + window_size]).argmax()
        window_labels.append(window_label)
    return np.array(window_labels)

window_labels = assign_labels_to_windows(labels, window_size, step_size)


In [6]:
#split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(reshaped_data, window_labels, test_size=0.2, random_state=21)

In [7]:
#compile the HART model
MobileHART.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [8]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=5,        
    restore_best_weights=True  
)

In [9]:
# fit the model on your training data
history = MobileHART.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test),callbacks=[early_stopping])

Epoch 1/100


2024-02-28 23:26:48.451280: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-02-28 23:26:49.223812: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-28 23:26:49.320590: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-28 23:26:50.926796: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f8d08584770 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-28 23:26:50.926814: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-02-28 23:26:50.926818: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (1): Quadro P6000, Compute Capability 6.1
2024-02-28 23:26:50.929725: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] d

1065/1065 [==============================] - 98s 60ms/step - loss: 1.4548 - accuracy: 0.5077 - val_loss: 1.3396 - val_accuracy: 0.5424
Epoch 2/100
1065/1065 [==============================] - 60s 57ms/step - loss: 0.8189 - accuracy: 0.7143 - val_loss: 0.6898 - val_accuracy: 0.7683
Epoch 3/100
1065/1065 [==============================] - 60s 57ms/step - loss: 0.6409 - accuracy: 0.7760 - val_loss: 0.8312 - val_accuracy: 0.7358
Epoch 4/100
1065/1065 [==============================] - 61s 57ms/step - loss: 0.5576 - accuracy: 0.8064 - val_loss: 0.6167 - val_accuracy: 0.8054
Epoch 5/100
1065/1065 [==============================] - 62s 58ms/step - loss: 0.5010 - accuracy: 0.8293 - val_loss: 0.4906 - val_accuracy: 0.8413
Epoch 6/100
1065/1065 [==============================] - 60s 57ms/step - loss: 0.4584 - accuracy: 0.8467 - val_loss: 0.3247 - val_accuracy: 0.8944
Epoch 7/100
1065/1065 [==============================] - 61s 57ms/step - loss: 0.4087 - accuracy: 0.8620 - val_loss: 0.3806 - val_

In [10]:
# evaluate the model on the test data
performance = MobileHART.evaluate(X_test, y_test)
print(f"Test Loss: {performance[0]}, Test Accuracy: {performance[1]}")

267/267 [==============================] - 3s 12ms/step - loss: 0.2343 - accuracy: 0.9253
Test Loss: 0.2342822253704071, Test Accuracy: 0.9252995252609253


In [12]:
# predict classes on the test data
y_pred_prob = MobileHART.predict(X_test)  # This gives you the probabilities
y_pred = np.argmax(y_pred_prob, axis=1)  # Convert probabilities to class labels

#generate the classification report
report = classification_report(y_test, y_pred, target_names=[f"Class {i}" for i in range(activityCount)])

print(report)


267/267 [==============================] - 3s 12ms/step
              precision    recall  f1-score   support

     Class 0       0.86      0.94      0.90       242
     Class 1       0.90      0.87      0.89       277
     Class 2       0.98      0.99      0.98       263
     Class 3       0.98      1.00      0.99       241
     Class 4       0.84      0.84      0.84       241
     Class 5       0.89      0.94      0.91       235
     Class 6       0.98      0.92      0.95       263
     Class 7       0.94      0.99      0.96       262
     Class 8       1.00      0.95      0.97       257
     Class 9       0.92      0.89      0.91       262
    Class 10       0.85      0.90      0.87       261
    Class 11       1.00      0.96      0.98       259
    Class 12       0.97      0.99      0.98       278
    Class 13       0.99      0.97      0.98       282
    Class 14       0.97      0.96      0.96       245
    Class 15       0.94      0.92      0.93       243
    Class 16       0.92  

## Testing the healthy trained model on PD data

In [14]:
#read and preprocess the test data
test_columns = ["AccelX(g)_leg", "AccelY(g)_leg", "AccelZ(g)_leg", "GyroX(¬∞/s)_leg", "GyroY(¬∞/s)_leg", "GyroZ(¬∞/s)_leg", "AccelX(g)_hand", "AccelY(g)_hand", "AccelZ(g)_hand", "GyroX(¬∞/s)_hand", "GyroY(¬∞/s)_hand", "GyroZ(¬∞/s)_hand", "AccelX(g)_chest", "AccelY(g)_chest", "AccelZ(g)_chest", "Sample(V)", "Activity_Label"]
test_df = pd.read_csv("/home/jupyter-saswar/PD_smote.csv", header=0, names=test_columns)

#factorize test labels
test_labels = pd.factorize(test_df['Activity_Label'])[0]

#select relevant columns for test features
test_feature_columns = test_df.columns[:-1]  
test_data = test_df[test_feature_columns].values

test_reshaped_data = create_overlapping_windows(test_data, window_size, step_size, num_features)

test_window_labels = assign_labels_to_windows(test_labels, window_size, step_size)

test_performance = MobileHART.evaluate(test_reshaped_data, test_window_labels)
print(f"Test Loss: {test_performance[0]}, Test Accuracy: {test_performance[1]}")


1346/1346 [==============================] - 17s 12ms/step - loss: 12.6001 - accuracy: 0.0312
Test Loss: 12.600137710571289, Test Accuracy: 0.031237661838531494


In [15]:


#evaluate the model on the test data
test_predictions = MobileHART.predict(test_reshaped_data)
test_predictions = np.argmax(test_predictions, axis=1)

#print classification report
print(classification_report(test_window_labels, test_predictions))




1346/1346 [==============================] - 16s 12ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1388
           1       0.00      0.01      0.00      1389
           2       0.00      0.00      0.00      1389
           3       0.02      0.02      0.02      1389
           4       0.04      0.04      0.04      1387
           5       0.00      0.00      0.00      1390
           6       0.03      0.01      0.01      1389
           7       0.00      0.00      0.00      1389
           8       0.00      0.00      0.00      1389
           9       0.00      0.00      0.00      1389
          10       0.07      0.12      0.08      1387
          11       0.05      0.00      0.01      1390
          12       0.00      0.00      0.00      1390
          13       0.00      0.00      0.00      1390
          14       0.00      0.00      0.00      1389
          15       0.00      0.00      0.00      1389
          16       0.0

/home/jupyter-saswar/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jupyter-saswar/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jupyter-saswar/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [16]:
#calculate ratios and generate a DataFrame for detailed analysis

label_counts = defaultdict(int)
total_counts = defaultdict(int)

for i in range(len(test_window_labels)):
    actual_label = test_window_labels[i]
    predicted_label = test_predictions[i]
    label_counts[f"actual_{actual_label}_predicted_{predicted_label}"] += 1
    total_counts[f"actual{actual_label}"] += 1

data = []
for label, count in label_counts.items():
    actual_label = label.split("_")[1]
    total = total_counts[f"actual{actual_label}"]
    ratio = count / total if total > 0 else 0
    data.append([label, count, total, ratio])

results_df = pd.DataFrame(data, columns=["Label", "Count", "Total", "Ratio"])
results_df.sort_values(by="Ratio", ascending=False, inplace=True)
print(results_df)


                      Label  Count  Total     Ratio
201  actual_17_predicted_23   1389   1389  1.000000
216  actual_19_predicted_32   1388   1388  1.000000
287  actual_25_predicted_17   1388   1389  0.999280
121   actual_9_predicted_14   1387   1389  0.998560
195   actual_16_predicted_5   1387   1389  0.998560
..                      ...    ...    ...       ...
348  actual_11_predicted_31      1   1390  0.000719
319  actual_13_predicted_17      1   1390  0.000719
158  actual_13_predicted_32      1   1390  0.000719
228   actual_20_predicted_0      1   1390  0.000719
343  actual_12_predicted_32      1   1390  0.000719

[354 rows x 4 columns]


In [21]:
print('''Tweaked Classification Report for MobileHART
      trained on Non PD - Tested on PD''')
results_df.head(20)

Tweaked Classification Report for MobileHART
      trained on Non PD - Tested on PD


,Label,Count,Total,Ratio
201,actual_17_predicted_23,1389,1389,1.000000
216,actual_19_predicted_32,1388,1388,1.000000
287,actual_25_predicted_17,1388,1389,0.999280
121,actual_9_predicted_14,1387,1389,0.998560
195,actual_16_predicted_5,1387,1389,0.998560
284,actual_24_predicted_2,1386,1389,0.997840
289,actual_26_predicted_19,1385,1389,0.997120
111,actual_8_predicted_3,1378,1389,0.992081
204,actual_18_predicted_22,1353,1389,0.974082
147,actual_12_predicted_22,1330,1390,0.956835


## Mobile HART on PD Smote Data

In [23]:
columns = ["AccelX(g)_leg", "AccelY(g)_leg", "AccelZ(g)_leg", "GyroX(¬∞/s)_leg", "GyroY(¬∞/s)_leg", "GyroZ(¬∞/s)_leg", "AccelX(g)_hand", "AccelY(g)_hand", "AccelZ(g)_hand", "GyroX(¬∞/s)_hand", "GyroY(¬∞/s)_hand", "GyroZ(¬∞/s)_hand", "AccelX(g)_chest", "AccelY(g)_chest", "AccelZ(g)_chest", "Sample(V)", "Activity_Label"]
df1 = pd.read_csv("/home/jupyter-saswar/PD_smote.csv", header=0, names=columns)

labels = pd.factorize(df1['Activity_Label'])[0]

feature_columns = df1.columns[:-1] 
data = df1[feature_columns].values

In [24]:
#define parameters for overlapping windows
window_size = 128 
step_size = 64    
num_features = 16 

reshaped_data = create_overlapping_windows(data, window_size, step_size, num_features)

window_labels = assign_labels_to_windows(labels, window_size, step_size)

input_shape = (128, 16)
activityCount = len(np.unique(labels))  # Update this based on the unique number of labels


In [26]:
MHART = model.mobileHART_XS(input_shape, activityCount)

X_train, X_test, y_train, y_test = train_test_split(reshaped_data, window_labels, test_size=0.2, random_state=42)

MHART.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)


In [27]:

history = MHART.fit(X_train, y_train, epochs=200, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stopping])


Epoch 1/200
539/539 [==============================] - 63s 64ms/step - loss: 1.0552 - accuracy: 0.6530 - val_loss: 3.0530 - val_accuracy: 0.2346
Epoch 2/200
539/539 [==============================] - 31s 58ms/step - loss: 0.3515 - accuracy: 0.8915 - val_loss: 0.2242 - val_accuracy: 0.9331
Epoch 3/200
539/539 [==============================] - 31s 57ms/step - loss: 0.2392 - accuracy: 0.9259 - val_loss: 0.2273 - val_accuracy: 0.9200
Epoch 4/200
539/539 [==============================] - 31s 57ms/step - loss: 0.2053 - accuracy: 0.9390 - val_loss: 0.1314 - val_accuracy: 0.9616
Epoch 5/200
539/539 [==============================] - 31s 57ms/step - loss: 0.1693 - accuracy: 0.9492 - val_loss: 0.1818 - val_accuracy: 0.9477
Epoch 6/200
539/539 [==============================] - 31s 57ms/step - loss: 0.1376 - accuracy: 0.9584 - val_loss: 0.1813 - val_accuracy: 0.9493
Epoch 7/200
539/539 [==============================] - 31s 57ms/step - loss: 0.1557 - accuracy: 0.9542 - val_loss: 0.1137 - val_ac

In [28]:
# Evaluate the model on the test data
performance = MHART.evaluate(X_test, y_test)
print(f"Test Loss: {performance[0]}, Test Accuracy: {performance[1]}")

270/270 [==============================] - 4s 13ms/step - loss: 0.0775 - accuracy: 0.9769
Test Loss: 0.07753639668226242, Test Accuracy: 0.9768927097320557


In [29]:
test_predictions = MHART.predict(test_reshaped_data)
test_predictions = np.argmax(test_predictions, axis=1)

print(classification_report(test_window_labels, test_predictions))


1346/1346 [==============================] - 17s 12ms/step
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      1388
           1       0.98      0.97      0.97      1389
           2       0.97      0.94      0.96      1389
           3       0.99      0.99      0.99      1389
           4       0.96      0.91      0.93      1387
           5       0.99      1.00      0.99      1390
           6       0.98      0.99      0.99      1389
           7       0.98      0.98      0.98      1389
           8       0.98      1.00      0.99      1389
           9       1.00      0.98      0.99      1389
          10       0.95      0.96      0.96      1387
          11       1.00      1.00      1.00      1390
          12       1.00      1.00      1.00      1390
          13       0.93      0.98      0.95      1390
          14       0.98      1.00      0.99      1389
          15       0.98      1.00      0.99      1389
          16       1.0

## Mobile HART Trained on non PD and finetuned on 30% PD Data 

In [33]:
columns = ["AccelX(g)_leg", "AccelY(g)_leg", "AccelZ(g)_leg", "GyroX(¬∞/s)_leg", "GyroY(¬∞/s)_leg", "GyroZ(¬∞/s)_leg", "AccelX(g)_hand", "AccelY(g)_hand", "AccelZ(g)_hand", "GyroX(¬∞/s)_hand", "GyroY(¬∞/s)_hand", "GyroZ(¬∞/s)_hand", "AccelX(g)_chest", "AccelY(g)_chest", "AccelZ(g)_chest", "Sample(V)", "Activity_Label"]
df = pd.read_csv("/home/jupyter-saswar/HealthySubs_Modeling/healthy_smote.csv", header=0, names=columns)

labels = pd.factorize(df['Activity_Label'])[0]

feature_columns = df.columns[:-1]  
data = df[feature_columns].values

In [34]:
window_size = 128 
step_size = 64    
num_features = 16

reshaped_data = create_overlapping_windows(data, window_size, step_size, num_features)

window_labels = assign_labels_to_windows(labels, window_size, step_size)

input_shape = (128, 16)
activityCount = len(np.unique(labels))  

MHART = model.mobileHART_XS(input_shape, activityCount)

X_train, X_test, y_train, y_test = train_test_split(reshaped_data, window_labels, test_size=0.2, random_state=42)

MHART.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

history = MHART.fit(X_train, y_train, epochs=200, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stopping])


Epoch 1/200
533/533 [==============================] - 58s 61ms/step - loss: 1.4851 - accuracy: 0.5028 - val_loss: 3.2631 - val_accuracy: 0.1885
Epoch 2/200
533/533 [==============================] - 31s 58ms/step - loss: 0.6964 - accuracy: 0.7585 - val_loss: 0.7727 - val_accuracy: 0.7317
Epoch 3/200
533/533 [==============================] - 31s 58ms/step - loss: 0.5333 - accuracy: 0.8177 - val_loss: 0.6333 - val_accuracy: 0.7903
Epoch 4/200
533/533 [==============================] - 31s 58ms/step - loss: 0.4712 - accuracy: 0.8412 - val_loss: 0.5126 - val_accuracy: 0.8346
Epoch 5/200
533/533 [==============================] - 31s 58ms/step - loss: 0.4105 - accuracy: 0.8633 - val_loss: 0.3693 - val_accuracy: 0.8767
Epoch 6/200
533/533 [==============================] - 31s 58ms/step - loss: 0.3542 - accuracy: 0.8804 - val_loss: 0.5008 - val_accuracy: 0.8419
Epoch 7/200
533/533 [==============================] - 31s 58ms/step - loss: 0.3247 - accuracy: 0.8918 - val_loss: 0.6939 - val_ac

In [36]:
performance = MHART.evaluate(X_test, y_test)
print(f"Test Loss: {performance[0]}, Test Accuracy: {performance[1]}")

267/267 [==============================] - 3s 12ms/step - loss: 0.1946 - accuracy: 0.9433
Test Loss: 0.19458618760108948, Test Accuracy: 0.9432699084281921


In [37]:
columns = ["AccelX(g)_leg","AccelY(g)_leg","AccelZ(g)_leg","GyroX(¬∞/s)_leg","GyroY(¬∞/s)_leg","GyroZ(¬∞/s)_leg","AccelX(g)_hand","AccelY(g)_hand","AccelZ(g)_hand","GyroX(¬∞/s)_hand","GyroY(¬∞/s)_hand","GyroZ(¬∞/s)_hand","AccelX(g)_chest","AccelY(g)_chest","AccelZ(g)_chest","Sample(V)","Activity_Label"]
df = pd.read_csv("/home/jupyter-saswar/PD_smote.csv", header=0, names = columns)

from sklearn.preprocessing import LabelEncoder

LE = LabelEncoder()
df['Activity_Label'] = LE.fit_transform(df['Activity_Label'])

In [38]:

features = df.columns[:-1]  
labels_encoded = df['Activity_Label'].values

data_filtered = df[features].values

window_size = 128
step_size = 64
num_features = 16

reshaped_data_filtered = create_overlapping_windows(data_filtered, window_size, step_size, num_features)

window_labels_filtered = assign_labels_to_windows(labels_encoded, window_size, step_size)

X_fine_tune, X_test_new, y_fine_tune, y_test_new = train_test_split(reshaped_data_filtered, window_labels_filtered, test_size=0.7, random_state=21)

# Fine-tune the model on the 30% subset
MHART.fit(X_fine_tune, y_fine_tune, epochs=200, batch_size=64, validation_split=0.2, callbacks=[early_stopping])


Epoch 1/200
162/162 [==============================] - 12s 74ms/step - loss: 0.9323 - accuracy: 0.7824 - val_loss: 0.2360 - val_accuracy: 0.9342
Epoch 2/200
162/162 [==============================] - 9s 58ms/step - loss: 0.2474 - accuracy: 0.9256 - val_loss: 0.1890 - val_accuracy: 0.9404
Epoch 3/200
162/162 [==============================] - 9s 58ms/step - loss: 0.1763 - accuracy: 0.9466 - val_loss: 0.1677 - val_accuracy: 0.9512
Epoch 4/200
162/162 [==============================] - 9s 58ms/step - loss: 0.1516 - accuracy: 0.9529 - val_loss: 0.1633 - val_accuracy: 0.9524
Epoch 5/200
162/162 [==============================] - 10s 59ms/step - loss: 0.1220 - accuracy: 0.9628 - val_loss: 0.1269 - val_accuracy: 0.9621
Epoch 6/200
162/162 [==============================] - 9s 58ms/step - loss: 0.0926 - accuracy: 0.9696 - val_loss: 0.1221 - val_accuracy: 0.9690
Epoch 7/200
162/162 [==============================] - 9s 59ms/step - loss: 0.0886 - accuracy: 0.9721 - val_loss: 0.1199 - val_accurac

In [39]:

# Evaluate the model on the remaining 70%
performance_new = MHART.evaluate(X_test_new, y_test_new)
print(f"Test Loss: {performance_new[0]}, Test Accuracy: {performance_new[1]}")


942/942 [==============================] - 12s 13ms/step - loss: 0.1118 - accuracy: 0.9713
Test Loss: 0.11182010918855667, Test Accuracy: 0.9713006019592285


## Mobile HART Trained on non PD and finetuned on 80% PD Data 

In [40]:
MHART = model.mobileHART_XS(input_shape, activityCount)

X_train, X_test, y_train, y_test = train_test_split(reshaped_data, window_labels, test_size=0.2, random_state=42)

MHART.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

history = MHART.fit(X_train, y_train, epochs=200, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stopping])


Epoch 1/200
533/533 [==============================] - 58s 60ms/step - loss: 1.9507 - accuracy: 0.3254 - val_loss: 4.0618 - val_accuracy: 0.0608
Epoch 2/200
533/533 [==============================] - 31s 58ms/step - loss: 1.0031 - accuracy: 0.6495 - val_loss: 0.8818 - val_accuracy: 0.6905
Epoch 3/200
533/533 [==============================] - 31s 58ms/step - loss: 0.6869 - accuracy: 0.7574 - val_loss: 0.7851 - val_accuracy: 0.7550
Epoch 4/200
533/533 [==============================] - 31s 58ms/step - loss: 0.5667 - accuracy: 0.8021 - val_loss: 0.5874 - val_accuracy: 0.7952
Epoch 5/200
533/533 [==============================] - 31s 58ms/step - loss: 0.4772 - accuracy: 0.8352 - val_loss: 0.4976 - val_accuracy: 0.8284
Epoch 6/200
533/533 [==============================] - 31s 58ms/step - loss: 0.4427 - accuracy: 0.8492 - val_loss: 0.4916 - val_accuracy: 0.8288
Epoch 7/200
533/533 [==============================] - 31s 58ms/step - loss: 0.3997 - accuracy: 0.8627 - val_loss: 0.4220 - val_ac

In [42]:

features = df.columns[:-1]  
labels_encoded = df['Activity_Label'].values

data_filtered = df[features].values

window_size = 128
step_size = 64
num_features = 16

reshaped_data_filtered = create_overlapping_windows(data_filtered, window_size, step_size, num_features)

window_labels_filtered = assign_labels_to_windows(labels_encoded, window_size, step_size)

X_fine_tune, X_test_new, y_fine_tune, y_test_new = train_test_split(reshaped_data_filtered, window_labels_filtered, test_size=0.2, random_state=21)

# Fine-tune the model on the 30% subset
MHART.fit(X_fine_tune, y_fine_tune, epochs=200, batch_size=64, validation_split=0.2, callbacks=[early_stopping])


Epoch 1/200
431/431 [==============================] - 27s 64ms/step - loss: 0.5598 - accuracy: 0.8581 - val_loss: 0.2073 - val_accuracy: 0.9411
Epoch 2/200
431/431 [==============================] - 25s 58ms/step - loss: 0.1850 - accuracy: 0.9436 - val_loss: 0.1577 - val_accuracy: 0.9579
Epoch 3/200
431/431 [==============================] - 25s 58ms/step - loss: 0.1374 - accuracy: 0.9580 - val_loss: 0.1579 - val_accuracy: 0.9588
Epoch 4/200
431/431 [==============================] - 25s 58ms/step - loss: 0.1169 - accuracy: 0.9653 - val_loss: 0.1620 - val_accuracy: 0.9546
Epoch 5/200
431/431 [==============================] - 25s 58ms/step - loss: 0.0831 - accuracy: 0.9734 - val_loss: 0.0941 - val_accuracy: 0.9762
Epoch 6/200
431/431 [==============================] - 25s 58ms/step - loss: 0.0951 - accuracy: 0.9717 - val_loss: 0.1533 - val_accuracy: 0.9608
Epoch 7/200
431/431 [==============================] - 25s 58ms/step - loss: 0.0851 - accuracy: 0.9739 - val_loss: 0.0963 - val_ac

In [43]:

# Evaluate the model on the remaining 20%
performance_new = MHART.evaluate(X_test_new, y_test_new)
print(f"Test Loss: {performance_new[0]}, Test Accuracy: {performance_new[1]}")


270/270 [==============================] - 3s 12ms/step - loss: 0.0842 - accuracy: 0.9769
Test Loss: 0.0842008963227272, Test Accuracy: 0.9768927097320557
